# Comparison of GSPHAR and GARCH Models

This notebook compares the performance of the trained GSPHAR model with a traditional GARCH model for volatility forecasting.

In [ ]:
# Import necessary libraries
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import torch
from arch import arch_model
from sklearn.metrics import mean_squared_error, mean_absolute_error
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Add the parent directory to the path to import from the GSPHAR package
sys.path.insert(0, os.path.abspath('..'))

# Import from local modules
from config import settings
from src.data import load_data, split_data, create_lagged_features, prepare_data_dict, create_dataloaders
from src.models import GSPHAR
from src.utils.graph_utils import compute_spillover_index
from src.utils.model_utils import load_model

# Set plotting style
plt.style.use('ggplot')
sns.set(style="darkgrid")
%matplotlib inline

## 1. Load and Prepare Data

In [ ]:
# Load data
data_file = '../data/rv5_sqrt_24.csv'  # Use relative path from notebooks directory
data = load_data(data_file)
print(f"Data shape: {data.shape}")
data.head()

In [ ]:
# Split data into train and test sets
train_split = 0.7
train_dataset_raw, test_dataset_raw = split_data(data, train_split)
print(f"Train data shape: {train_dataset_raw.shape}")
print(f"Test data shape: {test_dataset_raw.shape}")

## 2. Load Trained GSPHAR Model

In [ ]:
# Set parameters
h = settings.PREDICTION_HORIZON
look_back = settings.LOOK_BACK_WINDOW
input_dim = settings.INPUT_DIM
output_dim = settings.OUTPUT_DIM
filter_size = settings.FILTER_SIZE

# Get market indices
market_indices_list = train_dataset_raw.columns.tolist()

# Compute spillover index
DY_adj = compute_spillover_index(train_dataset_raw, h, look_back, 0.0, standardized=True)

# Create lagged features
train_dataset = create_lagged_features(train_dataset_raw, market_indices_list, h, look_back)
test_dataset = create_lagged_features(test_dataset_raw, market_indices_list, h, look_back)

# Prepare data dictionaries
train_dict = prepare_data_dict(train_dataset, market_indices_list, look_back)
test_dict = prepare_data_dict(test_dataset, market_indices_list, look_back)

# Create dataloaders
batch_size = settings.BATCH_SIZE
dataloader_train, dataloader_test = create_dataloaders(train_dict, test_dict, batch_size)

In [ ]:
# Create model
model = GSPHAR(input_dim, output_dim, filter_size, DY_adj)

# Load trained model
model_save_name = settings.MODEL_SAVE_NAME_PATTERN.format(
    filter_size=filter_size,
    h=h
)
# Make sure we're using the correct path
model_path = os.path.join('..', 'models', f'{model_save_name}.tar')
if os.path.exists(model_path):
    checkpoint = torch.load(model_path, map_location='cpu')
    model.load_state_dict(checkpoint['model_state_dict'])
    mae_loss = checkpoint['loss']
    print(f"Loaded model: {model_save_name}")
    print(f"MAE loss: {mae_loss:.4f}")
    trained_model = model
else:
    print(f"Model file not found: {model_path}")
    print("Available models:")
    for model_file in os.listdir(os.path.join('..', 'models')):
        print(f"  - {model_file}")

## 3. Generate GSPHAR Predictions

In [ ]:
def generate_gsphar_predictions(model, dataloader, market_indices_list, test_dates=None):
    """Generate predictions using the GSPHAR model."""
    model.eval()
    all_predictions = []
    all_actuals = []
    
    with torch.no_grad():
        for batch in dataloader:
            x_lag1, x_lag5, x_lag22, y = batch
            
            # Move to CPU for consistent comparison
            x_lag1 = x_lag1.cpu()
            x_lag5 = x_lag5.cpu()
            x_lag22 = x_lag22.cpu()
            
            # Generate predictions
            output, _, _ = model(x_lag1, x_lag5, x_lag22)
            
            # Store predictions and actuals
            all_predictions.append(output.cpu().numpy())
            all_actuals.append(y.numpy())
    
    # Concatenate all predictions and actuals
    all_predictions = np.vstack(all_predictions)
    all_actuals = np.vstack(all_actuals)
    
    # Create DataFrames with proper datetime index if provided
    if test_dates is not None:
        # Make sure we have the right number of dates
        if len(test_dates) >= len(all_predictions):
            # Use the last portion of test_dates that matches our prediction length
            dates_to_use = test_dates[-len(all_predictions):]
            pred_df = pd.DataFrame(all_predictions, index=dates_to_use, columns=market_indices_list)
            actual_df = pd.DataFrame(all_actuals, index=dates_to_use, columns=market_indices_list)
        else:
            print("Warning: Not enough dates provided for predictions. Using default index.")
            pred_df = pd.DataFrame(all_predictions, columns=market_indices_list)
            actual_df = pd.DataFrame(all_actuals, columns=market_indices_list)
    else:
        pred_df = pd.DataFrame(all_predictions, columns=market_indices_list)
        actual_df = pd.DataFrame(all_actuals, columns=market_indices_list)
    
    return pred_df, actual_df

# Get the test dates from the test dataset
test_dates = test_dataset_raw.index

# Generate predictions with dates
gsphar_pred_df, gsphar_actual_df = generate_gsphar_predictions(trained_model, dataloader_test, market_indices_list, test_dates)

print("GSPHAR predictions shape:", gsphar_pred_df.shape)
print("GSPHAR actuals shape:", gsphar_actual_df.shape)
print("GSPHAR predictions index type:", type(gsphar_pred_df.index))

## 4. Implement GARCH Model

In [ ]:
def fit_garch_model(series, p=1, q=1):
    """Fit a GARCH(p,q) model to the given series."""
    model = arch_model(series, vol='Garch', p=p, q=q, rescale=False)
    result = model.fit(disp='off')
    return result

def generate_garch_predictions(train_data, test_data, market_indices_list, p=1, q=1, horizon=5):
    """Generate predictions using GARCH models for each market index."""
    predictions = {}
    actuals = {}
    models = {}
    
    # Get the test dates
    test_dates = test_data.index
    
    # For each market index
    for market_index in tqdm(market_indices_list, desc="Fitting GARCH models"):
        # Get the training data for this market index
        train_series = train_data[market_index]
        
        # Fit GARCH model
        model = fit_garch_model(train_series, p=p, q=q)
        models[market_index] = model
        
        # Generate forecasts
        forecasts = model.forecast(horizon=horizon, reindex=False)
        
        # Extract the h-step ahead variance forecast and take square root for volatility
        variance_forecast = forecasts.variance.iloc[-1, horizon-1]
        volatility_forecast = np.sqrt(variance_forecast)
        
        # Store prediction and actual
        predictions[market_index] = [volatility_forecast]
        actuals[market_index] = [test_data[market_index].iloc[0]]
        
        # For the rest of the test data, use rolling forecasts
        for i in range(1, len(test_data)):
            # Update the model with the latest observation
            updated_data = pd.concat([train_series, test_data[market_index].iloc[:i]])
            model = fit_garch_model(updated_data, p=p, q=q)
            
            # Generate forecast
            forecasts = model.forecast(horizon=horizon, reindex=False)
            variance_forecast = forecasts.variance.iloc[-1, horizon-1]
            volatility_forecast = np.sqrt(variance_forecast)
            
            # Store prediction and actual
            predictions[market_index].append(volatility_forecast)
            if i < len(test_data):
                actuals[market_index].append(test_data[market_index].iloc[i])
    
    # Convert to DataFrames with datetime index
    pred_df = pd.DataFrame(predictions, index=test_dates[:len(predictions[market_indices_list[0]])])
    actual_df = pd.DataFrame(actuals, index=test_dates[:len(actuals[market_indices_list[0]])])
    
    return pred_df, actual_df, models

In [ ]:
# Generate GARCH predictions for a subset of indices to save time
# You can increase this number or use all indices if you have time
subset_indices = market_indices_list[:3]  # Use first 3 indices
print(f"Using subset of indices for GARCH: {subset_indices}")

garch_pred_df, garch_actual_df, garch_models = generate_garch_predictions(
    train_dataset_raw[subset_indices], 
    test_dataset_raw[subset_indices], 
    subset_indices,
    p=1, q=1, horizon=h
)

print("GARCH predictions shape:", garch_pred_df.shape)
print("GARCH actuals shape:", garch_actual_df.shape)

## 5. Compare Model Performance

In [ ]:
def calculate_metrics(predictions, actuals, market_indices):
    """Calculate performance metrics for each market index."""
    metrics = {}
    
    for market_index in market_indices:
        y_pred = predictions[market_index]
        y_true = actuals[market_index]
        
        # Calculate metrics
        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_true, y_pred)
        
        metrics[market_index] = {
            'MSE': mse,
            'RMSE': rmse,
            'MAE': mae
        }
    
    return metrics

In [ ]:
# Calculate metrics for GSPHAR model
gsphar_metrics = calculate_metrics(
    gsphar_pred_df[subset_indices], 
    gsphar_actual_df[subset_indices], 
    subset_indices
)

# Calculate metrics for GARCH model
garch_metrics = calculate_metrics(
    garch_pred_df, 
    garch_actual_df, 
    subset_indices
)

# Display metrics
print("GSPHAR Model Metrics:")
for market_index, metrics in gsphar_metrics.items():
    print(f"\n{market_index}:")
    for metric_name, value in metrics.items():
        print(f"  {metric_name}: {value:.4f}")

print("\n" + "-"*50 + "\n")

print("GARCH Model Metrics:")
for market_index, metrics in garch_metrics.items():
    print(f"\n{market_index}:")
    for metric_name, value in metrics.items():
        print(f"  {metric_name}: {value:.4f}")

## 6. Visualize Predictions

In [ ]:
def plot_predictions(gsphar_pred, gsphar_actual, garch_pred, garch_actual, market_index):
    """Plot predictions from both models for a given market index using Plotly."""
    import pandas as pd
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    
    # Set Plotly as the backend for pandas plotting
    pd.options.plotting.backend = "plotly"
    
    # Get the dates from both actual datasets
    gsphar_dates = gsphar_actual.index
    garch_dates = garch_actual.index
    
    # Find common date range
    common_dates = gsphar_dates.intersection(garch_dates)
    
    if len(common_dates) == 0:
        # If no common dates, use the test dataset dates
        print("Warning: No common dates found between GSPHAR and GARCH predictions.")
        print("Using GSPHAR dates for plotting.")
        
        # Create separate figures for each model
        fig = go.Figure()
        
        # Add GSPHAR data
        fig.add_trace(go.Scatter(
            x=gsphar_dates,
            y=gsphar_actual[market_index],
            mode='lines',
            name='Actual'
        ))
        
        fig.add_trace(go.Scatter(
            x=gsphar_dates,
            y=gsphar_pred[market_index],
            mode='lines',
            name='GSPHAR'
        ))
        
        # Add GARCH data with secondary x-axis
        fig.add_trace(go.Scatter(
            x=garch_dates,
            y=garch_pred[market_index],
            mode='lines',
            name='GARCH'
        ))
    else:
        # Filter data to common date range
        gsphar_actual_common = gsphar_actual.loc[common_dates, market_index]
        gsphar_pred_common = gsphar_pred.loc[common_dates, market_index]
        garch_pred_common = garch_pred.loc[common_dates, market_index]
        
        # Create DataFrame with aligned data
        df_plot = pd.DataFrame({
            'Actual': gsphar_actual_common,
            'GSPHAR': gsphar_pred_common,
            'GARCH': garch_pred_common
        }, index=common_dates)
        
        # Create the plot
        fig = df_plot.plot(
            title=f'Volatility Predictions for {market_index}',
            labels=dict(index="Date", value="Volatility"),
            template="plotly_white"
        )
    
    # Update layout for better appearance
    fig.update_layout(
        height=600,
        width=1000,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        hovermode="x unified",
        title={
            'text': f'Volatility Predictions for {market_index}',
            'x': 0.5,
            'xanchor': 'center'
        }
    )
    
    # Add grid
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey',
                    tickangle=45, tickformat='%Y-%m-%d')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
    
    # Show the plot
    fig.show()

In [ ]:
# Plot predictions for each market index in the subset
for market_index in subset_indices:
    plot_predictions(
        gsphar_pred_df, 
        gsphar_actual_df, 
        garch_pred_df, 
        garch_actual_df, 
        market_index
    )

## 7. Conclusion

In this notebook, we compared the performance of the GSPHAR model with a traditional GARCH model for volatility forecasting. The comparison was based on several metrics including MSE, RMSE, and MAE.

Key findings:
1. [Add your observations about which model performed better]
2. [Add insights about the strengths and weaknesses of each model]
3. [Add any other relevant conclusions]

Future work could include:
1. Testing with different GARCH specifications (e.g., EGARCH, GJR-GARCH)
2. Extending the comparison to more market indices
3. Implementing a hybrid model that combines the strengths of both approaches